In [ ]:
import tensorflow as tf
import numpy as np
from __future__ import division, print_function, absolute_import


#for plotting
import matplotlib.pyplot as plt

In [ ]:
#importing mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
#setting parameters for learning
lr = 0.01
num_epochs = 30
batch_size = 256
display_step = 2

#for plotting
examples_to_show = 10

In [ ]:
#setting parameters for network layers
n_input = 784 #28x28
n_hidden_1 = 256
n_hidden_2 = 128

In [ ]:
#setting up placeholder input and output
X = tf.placeholder("float", [None, n_input])

In [ ]:
#setting up layer sizes

weights = {
    'encoder_1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_2': tf.Variable(tf.random_normal([n_hidden_1, n_input]))
}

biases = {
    'encoder_1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_2': tf.Variable(tf.random_normal([n_input]))
}

In [ ]:
#setting up the network structure

def encoder(x):
    activation_1 = tf.nn.sigmoid(tf.matmul(x, weights['encoder_1']) + biases['encoder_1'])
    activation_2 = tf.nn.sigmoid(tf.matmul(activation_1, weights['encoder_2']) + biases['encoder_2'])
    return activation_2

def decoder(x):
    activation_1 = tf.nn.sigmoid(tf.matmul(x, weights['decoder_1']) + biases['decoder_1'])
    activation_2 = tf.nn.sigmoid(tf.matmul(activation_1, weights['decoder_2']) + biases['decoder_2'])
    return activation_2

In [ ]:
#defining output from the network and supervisory labels (input data itself)
decoded = decoder(encoder(X))
actual = X

In [ ]:
#defining loss and optimizer
cost = tf.reduce_mean(tf.pow((actual - decoded), 2)) #L2 norm
optimizer = tf.train.RMSPropOptimizer(lr).minimize(cost)

In [ ]:
#initializing all the variables
init = tf.global_variables_initializer()

In [ ]:
#running the graph
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(mnist.train.num_examples / batch_size)
    
    #training
    for epoch in range(num_epochs):
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            #add noise to x (gaussian)
            noise = tf.random_normal(shape=[batch_size, 784]).eval()
            batch_xs = batch_xs + (noise)/5
            
            #optimizing w/ backprop and running cost op (for loss)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        
        #showing logs per epoch
        if (epoch % display_step) == 0:
            print('Epoch=', epoch+1, ' ; cost=', c)
            
    print("Optimization Done")
    
    
    #running net on test set
    encode_decode = sess.run(decoded, feed_dict={X:mnist.test.images[:examples_to_show]})
    
    #plot input images vs output images
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(examples_to_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
        a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
        
    f.show()
    plt.draw()
    plt.show()